# <center>Основні метрики якості класифікації 

### Матриця невідповідностей

Існує багато метрик, що дають змогу обчислити кількісну характеристику бінарного класифікатора. У випадку віднесення одного об'єкта до одного з двох класів можливі 4 результати. Їх зручно відображати за допомогою матриці невідповідностей (з англ. "[confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix)"):

- $TP$ = кількість правильно класифікованих цільових об'єктів;
- $FP$ = кількість нецільових об'єктів, що класифіковані як цільові (помилки першого роду);
- $TN$ = кількість правильно класифікованих нецільових прикладів;
- $FN$ = кількість цільових об'єктів, що класифіковані як нецільові (помилки другого роду).

<center>
<img src="../../img/contingency.png" width = "500">
</center>

Отримати таку таблицю можна за допомогою функції [sklearn.metrics.confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html). Необхідно їй на вхід передати дійсні та передбачені класифікатором мітки.

In [ ]:
%matplotlib inline
import numpy as np
import seaborn as sns
from sklearn import metrics

true_labels = np.array([0, 1, 0, 0, 1, 1, 1, 1])
predicted_labels = np.array([0, 1, 1, 0, 0, 1, 0, 0])

my_metrics = metrics.confusion_matrix(true_labels, predicted_labels)
print(my_metrics)

### Характеристики бінарного класифікатора

З огляду на таблицю вище, введемо кілька величин, що кількісно характеризують бінарний класифікатор:

$$Recall = TPR = \frac{TP}{TP + FN},$$


$$\quad SPC = \frac{TN}{TN + FP},$$


$$\quad Precision = PPV = \frac{TP}{TP + FP},$$


$$\quad FPR = 1 - SPC,$$


$$ACC = \frac{TP + TN}{TP + TN + FP + FN},$$


$$\quad F1 = 2\frac{PPV\cdot TRP}{PPV + TPR}.$$


Повнота $TPR$ (True positive rate, recall, sensitivity) - частка правильно класифікованих цільових об'єктів поміж усіх цільових об'єктів.

Специфічність $SPC$ (Specificity, true negative rate) - частка правильно класифікованих нецільових об'єктів поміж усіх нецільових об'єктів.

Влучність $PPV$ (Positive predictive value, precision) - частка правильно класифікованих цільових об'єктів поміж усіх об'єктів, що класифіковані праильно.

$FPR$ (False positive rate) - частка помилково класифікованих нецільових об'єктів поміж усіх нецільових об'єктів.

$ACC$ (Accuracy) - частка правильно класифікованих об'єктів поміж усіх об'єктів. $ACC$ є основною характеристикою якості класифікації.

$F1$ (F1-measure) - середнє гармонійне влучності та повноти; ця метрика дає змогу врахувати обидві характеристики одночасно.


In [ ]:
PPV = metrics.precision_score(true_labels, predicted_labels)
TPR = metrics.recall_score(true_labels, predicted_labels)
F1 = metrics.f1_score(true_labels, predicted_labels)
ACC = metrics.accuracy_score(true_labels, predicted_labels)
PPV, TPR, F1, ACC

### ROC-крива и AUC

Здебільшого бінарні класифікатори мають вигляд $a(x) = \mbox{sign}(f(x, w) - w_0)$, де $w, w_0$ - параметри алгоритму. Тобто спочатку будується розділяюча поверхня $f(x, w) = w_0 $, після чого об'єкти, що перебувають з однієї сторони від неї класифікуються як цільові, з іншої - як нецільові.

ROC-крива (Receiver Operating Characteristic) – це графічна характеристика якості бінарного класифікатора, що виражає залежність TPR від FPR при зміні порога вирішального правила. Ця крива ілюструє, якою буде якість класифікації за різних значень $w_0$ і фіксованому значенні $w$.

ROC-крива проходить через точки (0, 0) та (1, 1) і монотонно не зменшується. Чим ближче крива всередині квадрата $[0, 1]\times[0, 1]$ до лівого верхнього кута, тим краще. Ідеальний варіант - крива, що проходить через три точки: (0, 0), (1, 1) та (0, 1). Діагональ цього квадрата відповідає випадковій класифікації. Типова ROC-крива для класифікатора відповідає кривій на рисунку нижче.

<center>
<img src="../../img/ROC.jpg" width = "350">
</center>

На практиці ROC-криву завжди оцінюють за незалежною тестовою вибіркою, щоб уникнути перенавчання.

Площа під ROC-кривою AUC (Area Under Curve) є кількісною характеристикою якості класифікації, яка не залежить від співвідношення помилок. Чим більше значення AUC, тим "краще" є модель класифікації.

### Дисбаланс класів

На практиці у разі дисбалансу класів вдаються до таких [дій](http://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/):

- зібрати більше даних, особливо об'єктів рідкісного класу (не завжди можливо);
- використовувати методи, що ґрунтуються на деревах рішень – [випадковий ліс](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) або [градієнтний бустинг над деревами](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html); дерева не так схильні до проблеми дисбалансу класів;
- використовувати метрики типу F1, ROC AUC та [Cohen's kappa](https://en.wikipedia.org/wiki/Cohen%27s_kappa), а не ACC;
- використовувати метрику, в якій помилка на об'єкті з рідкісного класу входить із більшою вагою, ніж помилка на об'єкті з типового класу;
- застосовувати [oversampling](https://en.wikipedia.org/wiki/Oversampling_and_undersampling_in_data_analysis) та [undersampling](https://en.wikipedia.org/wiki/Oversampling_and_undersampling_in_data_analysis); у першому випадку до вибірки додаються об'єкти рідкісного класу (семплюються із заміщенням), у другому - об'єкти типового класу випадково видаляються з вибірки;
- згенерувати штучні об'єкти рідкісного класу, Synthetic Minority Over-sampling Technique ([SMOTE](https://imbalanced-learn.org/stable/over_sampling.html#smote-adasyn)); додаткова [реалізація](https://github.com/fmfn/UnbalancedDataset) на Python;
- розбити один великий клас на кілька менших та застосувати стратегії [One Vs. All](https://machinelearningmastery.com/one-vs-rest-and-one-vs-one-for-multi-class-classification/) або [One Vs. One](https://machinelearningmastery.com/one-vs-rest-and-one-vs-one-for-multi-class-classification/);
- застосувати алгоритми пошуку викидів або OneClass алгоритми (наприклад, [OneClass SVM](https://machinelearningmastery.com/one-class-classification-algorithms/)).

## Приклади

In [ ]:
import sys

from sklearn.svm import SVC, LinearSVC

if sys.version_info.major == 2:
    from urllib import urlopen
elif sys.version_info.major == 3:
    from urllib.request import urlopen

from sklearn import datasets
from sklearn.metrics import auc, roc_curve
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# Завантажуємо дані щодо діабету індіанців Піма зі сховища машинного навчання UCI
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/pima-indians-diabetes/pima-indians-diabetes.data"
raw_data = urlopen(url)
data = np.loadtxt(raw_data, delimiter=",")

X = data[:, :8]
y = data[:, 8]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Задаємо параметр регуляризації терміну помилки
C = 10.0

# Будуємо класифікатор Suppor Vector Machine
lin_svm = LinearSVC(C=C, dual=False).fit(X_train, y_train)
y_score = lin_svm.decision_function(X_test)

# Обчислюємо ROC-криву та ROC AUC
fpr, tpr, _ = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)

# Будуємо графік ROC-кривої для певного класу
plt.figure()
plt.plot(fpr, tpr, label="ROC curve (area = %0.2f)" % roc_auc)
plt.plot([0, 1], [0, 1], "k--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic example")
plt.legend(loc="lower right")
plt.show()

Вказані вище характеристики можна використовувати для підбору параметрів алгоритмів, наприклад, за допомогою крос-валідації. Знайдемо оптимальну з погляду $F_1$-міри кількість найближчих сусідів алгоритму $kNN$.

In [ ]:
knn = KNeighborsClassifier()
k_s = np.arange(1, 50, 2)

scores_f1 = list()

for k in k_s:
    knn.n_neighbors = k
    scores_f1.append(np.mean(cross_val_score(knn, X, y, scoring="f1")))

plt.figure(1, figsize=(10, 5))
plt.clf()
plt.plot(k_s, scores_f1, linewidth=2)
plt.axvline(k_s[np.argmax(scores_f1)], color="r")
plt.ylabel("F1-score")
plt.xlabel("Parameter k")
plt.xlim(1, 49)
plt.title("F1-optimal number of neighbors, $k$ = %d" % k_s[np.argmax(scores_f1)])
plt.show()

### Багатокласова класифікація

У разі, коли кількість класів більше двох, матриця невідповідностей визначається аналогічним чином: на перетині $i$-го рядка і $j$-го стовпця стоїть кількість об'єктів $i$-го класу, що відносяться класифікатором до класу $j$.

In [ ]:
true_labels = np.array([0, 1, 2, 0, 1, 2, 0, 1, 2])
predicted_labels = np.array([0, 2, 0, 2, 1, 0, 0, 1, 2])

my_metrics2 = metrics.confusion_matrix(true_labels, predicted_labels)
my_metrics2

### One vs. All

Багатокласова класифікація може бути зведена до бінарної у різний спосіб. Одним із них є підхід [One vs. All](https://developers.google.com/machine-learning/crash-course/multi-class-neural-networks/one-vs-all). Його суть в наступному: для кожного класу $i \in \{1, \dots, k\}$ навчимо бінарний класифікатор $a_i(x) = \mbox{sign}f_i(x)$ на початковій вибірці зі зміненими мітками (об'єкти $i$-го класу отримують мітку 1, всі об'єкти, що залишилися - мітку 0). Іншими словами, ми вчимо $a_i$ відрізняти $i$-ий клас від усіх інших. Після цього підсумковий класифікатор будується як $a(x) = \mbox{argmax}_{i \in \{1, \dots, k\}} f_i(x)$, тобто він видає клас з найбільшою оцінкою $f_i( x) $.

In [ ]:
iris = datasets.load_iris()
X, y = iris.data, iris.target

# Встановлення однієї чи всіх версій лінійної SVM
onevsall = OneVsRestClassifier(LinearSVC()).fit(X, y)
metrics.accuracy_score(y, onevsall.predict(X))

### Корисні посилання:

- [Статья](http://habrahabr.ru/post/228963/) "Как заставить работать бинарный классификатор чуточку лучше" на Habrahabr
- [ROC-кривая](http://www.machinelearning.ru/wiki/index.php?title=ROC-%D0%BA%D1%80%D0%B8%D0%B2%D0%B0%D1%8F)
- [Характеристики](https://en.wikipedia.org/wiki/Precision_and_recall) бинарного классификатора
- [One vs. All и One vs. One](https://en.wikipedia.org/wiki/Multiclass_classification)
- [Quora](https://www.quora.com/In-classification-how-do-you-handle-an-unbalanced-training-set) про несбаланированные выборки
- про несбаланированные выборки на [ресурсе](http://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/) Machine Learning Mastery